In [18]:
import pandas as pd

In [48]:
df = pd.read_csv('./data/final_2020.csv')
# df = pd.read_csv('./data/random100000.csv')

In [49]:
df['AIRLINE_COMPANY'].value_counts()[0]

1828375

***We are only considering the airlines which are having sufficient data so filtering the dataframe***

In [50]:
airline_vals = ['WN','AA','DL','UA','AS','B6','NK','G4']
airlines = ['WN','AA','DL','UA','AS','B6','NK','G4']

In [51]:
df = df[df['AIRLINE_COMPANY'].isin(airlines)]

In [52]:
df.columns

Index(['ITIN_ID', 'MARKET_COUPONS', 'QUARTER', 'ORIGIN', 'ORIGIN_WAC', 'DEST',
       'DEST_WAC', 'AIRPORT_GROUP', 'WAC_GROUP', 'DISTANCE_GROUP', 'MILES',
       'CONTINUE_USA', 'NUM_OF_TICKETS', 'AIRLINE_COMPANY',
       'PRICE_PER_TICKET'],
      dtype='object')

***Splitting the data into two parts one for training and one for testing.***

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
features = ['MARKET_COUPONS', 'QUARTER', 'ORIGIN', 'DEST', 'MILES', 'NUM_OF_TICKETS', 'AIRLINE_COMPANY']
X = df[features]
Y = df['PRICE_PER_TICKET']

In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=99)

In [55]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.fit_transform(X_test)
model = linear_model.LinearRegression()
model.fit(X_train_encoded, Y_train)


LinearRegression()

***As Mean squared error is very high in Liner regression, we will not be considering that model.***

In [ ]:
from sklearn.pipeline import make_pipeline
from category_encoders import TargetEncoder
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    OrdinalEncoder(),
    RandomForestRegressor(max_depth=20, n_estimators=20)
)

pipeline.fit(X_train, Y_train)

In [ ]:
model = pipeline.named_steps['randomforestregressor']
importances = pd.Series(model.feature_importances_, list(X.columns))

# Plot feature importances
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,6), facecolor = '#f0faf3')
ax.set(facecolor='#b4b4b4')
#plt.title('Feature Importances for 2018 Flights Data (Target: PricePerTicket)')
n=12
importances.sort_values()[-n:].plot.barh(color='#24ccff');

In [ ]:
import pickle
filename = './model/finalized_model_small.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test_encoded, Y_test)

In [ ]:
result

In [ ]:
from sklearn.metrics import r2_score
y_pred = pipeline.predict(X_test_encoded)
print('RandomForest R^2 Score = ', r2_score(Y_test, y_pred))

***As you can see r square score is less for random forest we will be finalizing that model***

In [ ]:
importances.sort_values()[-n:]

In [ ]:
importances = importances.sort_values()[-n:].index

In [ ]:
pipeline_encoder = pipeline.named_steps['ordinalencoder']
mapping = pipeline_encoder.mapping

origin_dict = dict(mapping[0])
dest_dict = mapping[1]
airline_dict = mapping[2]

In [ ]:
airline_dict["mapping"].to_csv("./LookupFiles/airline_dict.csv")

In [ ]:
dest_dict["mapping"].to_csv("./LookupFiles/dest_dict.csv")

In [ ]:
origin_dict["mapping"].to_csv("./LookupFiles/origin_dict.csv")

In [ ]:
origin_dict = pd.read_csv("./LookupFiles/origin_dict.csv").set_index('Unnamed: 0')
dest_dict = pd.read_csv("./LookupFiles/dest_dict.csv").set_index('Unnamed: 0')
airline_dict = pd.read_csv("./LookupFiles/airline_dict.csv").set_index('Unnamed: 0')

In [ ]:
origin_dict.loc['ECP', '0']

In [ ]:
import os
download_path = os.path.expanduser('~/data')
print(download_path)
